<td>   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a></td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/project_configuration/model_chat_evaluation_project.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/project_configuration/model_chat_evaluation_project.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Model Chat Evaluation Project Setup

This notebook will provide an example workflow of setting up a Model Chat Evaluation (MCE) Project with the Labelbox-Python SDK.
MOE Projects are set up differently then other editors with it's own unique method and modifications to existing methods:

- `client.create_model_evaluation_project`: Main method used to create a model chat evaluation project

- `client.create_ontology`: Methods used to create Labelbox ontologies for MCE project this requires a `ontology_kind` parameter set to `lb.OntologyKind.ModelEvaluation`

- `client.create_ontology_from_feature_schemas`: Similar to `client.create_ontology` but from a list of `feature schema ids` which is designed to allow you to use existing features instead of creating new features. This also requires a `ontology_kind` set to `lb.OntologyKind.ModelEvaluation`.

## Set Up

In [ ]:
%pip install -q "labelbox[data]"

In [ ]:
import labelbox as lb

## API Key and Client
Provide a valid API key below in order to properly connect to the Labelbox client. Please review [Create API key guide](https://docs.labelbox.com/reference/create-api-key) for more information.

In [ ]:
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Example: Create Model Chat Evaluation Project

The steps to creating a Model Chat Evaluation Project through the Labelbox-Python SDK are similar to creating a regular project. However, they vary slightly, and we will showcase the different methods in this example workflow.

### Create a MOE Ontology

You can create ontologies for Model Evaluation projects the same way as creating ontologies for other projects with the only requirement of passing in a `ontology_kind` parameter which needs set to `lb.OntologyKind.ModelEvaluation`. You can create ontologies with two methods: `client.create_ontology` and `client.create_ontology_from_feature_schemas`.

#### Option A: `client.create_ontology`

Typically, you create ontologies and generate the associated features at the same time. Below is an example of creating an ontology for your model chat evaluation project using supported tools and classifications. For information on supported annotation types visit our [model chat evaluation](https://docs.labelbox.com/docs/model-chat-evaluation#supported-annotation-types) guide.

In [ ]:
ontology_builder = lb.OntologyBuilder(
    tools=[
        lb.Tool(
            tool=lb.Tool.Type.MESSAGE_SINGLE_SELECTION,
            name="single select feature",
        ),
        lb.Tool(
            tool=lb.Tool.Type.MESSAGE_MULTI_SELECTION,
            name="multi select feature",
        ),
        lb.Tool(tool=lb.Tool.Type.MESSAGE_RANKING, name="ranking feature"),
    ],
    classifications=[
        lb.Classification(
            class_type=lb.Classification.Type.CHECKLIST,
            name="checklist feature",
            options=[
                lb.Option(value="option 1", label="option 1"),
                lb.Option(value="option 2", label="option 2"),
            ],
        ),
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            name="radio_question",
            options=[
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer"),
            ],
        ),
    ],
)

# Create ontology
ontology = client.create_ontology(
    "MCE ontology",
    ontology_builder.asdict(),
    media_type=lb.MediaType.Conversational,
    ontology_kind=lb.OntologyKind.ModelEvaluation,
)

### Option B: `client.create_ontology_from_feature_schemas`
Ontologies can also be created with feature schema IDs. This makes your ontologies with existing features compared to generating new features. You can get these features by going to the _Schema_ tab inside Labelbox. (uncomment the below code block for this option)

In [ ]:
# ontology = client.create_ontology_from_feature_schemas(
#     "MCE ontology",
#     feature_schema_ids=["<list of feature schema ids"],
#     media_type=lb.MediaType.Conversational,
#     ontology_kind=lb.OntologyKind.ModelEvaluation,
# )

### Set Up Model Chat Evaluation Project
You do not have to create data rows with a model evaluation project instead they are generated for you when you create the project. The method you use to create your project is `client.create_model_evaluation_project` which takes the same parameters as the traditional `client.create_project` but with a few specific additional parameters. 

__Setting up different models that you are evaluating can only be done inside the Labelbox platform__

#### Parameters
When using `client.create_model_evaluation_project` the following parameters are needed:

- `create_model_evaluation_project` parameters:

    - `name`: The name of your new project.

    - `description`: Optional description of your project.

    - `media_type`: The type of assets that this project will accept. This should be set to lb.MediaType.Conversational

    - `dataset_name`: The name of the dataset were the data rows that are generated will be located. Include this parameter only needed if wanting to create a new dataset.

    - `dataset_id`: A dataset ID of an existing Labelbox dataset. Include this parameter if you are wanting to append to an existing MOE dataset.

    - `data_row_count`: The number of data row assets that will be generated and used with your project.


In [ ]:
project = client.create_model_evaluation_project(
    name="Demo MCE Project",
    media_type=lb.MediaType.Conversational,
    dataset_name="Demo MCE dataset",
    data_row_count=100,
)

# Setup project with ontology created above
project.setup_editor(ontology)

## Exporting Model Chat Evaluation Project
Exporting from a Model Chat Evaluation project works the same as exporting from other projects. In this example, unless you have created labels inside the Labelbox platform your exported will be shown as empty. Please review our [Model Chat Evaluation Export](https://docs.labelbox.com/reference/export-model-chat-evaluation-annotations) guide for a sample export.

In [ ]:
# Start export from project
export_task = project.export()
export_task.wait_till_done()

# Conditional if task has errors
if export_task.has_errors():
    export_task.get_buffered_stream(stream_type=lb.StreamType.ERRORS).start(
        stream_handler=lambda error: print(error))

if export_task.has_result():
    # Start export stream
    stream = export_task.get_buffered_stream()

    # Iterate through data rows
    for data_row in stream:
        print(data_row.json)

## Clean Up

This section serves as an optional clean-up step to delete the Labelbox assets created within this guide. You will need to uncomment the delete methods shown.

In [ ]:
# project.delete()
# client.delete_unused_ontology(ontology.uid)
# dataset.delete()